In [56]:
import langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [57]:
# 加载文档并将其拆分成块
loader = TextLoader("../example_data/Elon Musk's Speech at WAIC 2023.txt",encoding='utf-8')
documents = loader.load()

In [58]:
# 将其拆分成块
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [59]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
db = Chroma.from_documents(docs, embeddings_model, persist_directory="chroma_db")

In [60]:
query = ("马斯克认为AI将对人类文明产生什么影响")
# 从磁盘加载
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embeddings_model)


In [61]:
# 检索
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke(query)
len(retrieved_docs)
print(retrieved_docs[0].page_content)

我认为人工智能在未来人类的演进中将发挥重要作用，并对文明产生深远的影响。

我们已经见证了数字计算能力的爆炸式增长。一个关键指标是数字计算机与生物大脑计算能力之比。这意味着人类与机器之间的算力比率不断增加，从而拉大了机器和生物之间的算力差距。

随着时间的推移，人工智能在整体智能中所占比例将逐渐降低，相对于机器智能而言，这将是一个根本性的深刻变化。尽管现在很难完全理解其影响，但这可能是人类历史上最为深刻的时期之一。特斯拉的Optimus人型机器人目前仍处于早期开发阶段，但未来我们将会看到更多的机器人。

因此，我们需要考虑另一个比例的问题，即机器人与人类的比例是多少？

目前看来，这个比例将超过1:1，也就是说全球机器人的数量将超过人类的数量。机器人的计算能力要远远超过人类，这似乎是一个发展趋势。这将带来积极的影响，同时也会带来一些消极的影响。积极方面是我们将进入一个没有短缺的时代，几乎任何你想要的东西都能够立即获得。由于未来世界中将存在大量机器人，它们的生产效率将远远超过人类，这是一个非常深刻的变化。

因此，我们需要小心确保最终结果对人类有益。


In [66]:
example_messages = prompt.invoke(
    {"context": "填充上下文", "question": "填充问题"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 填充问题 \nContext: 填充上下文 \nAnswer:")]

In [67]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 填充问题 
Context: 填充上下文 
Answer:


In [68]:
from langchain_core.prompts import PromptTemplate

template = """使用以下内容回答问题。
如果你不知道答案，就说你不知道，不要试图编造答案。
最多使用三句话，并尽可能简明扼要。
总是在回答的最后说“谢谢你的提问！”。

{context}

问题: {question}

有用的回答:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [69]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="qwen:1.8b")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | model
    | StrOutputParser()
)

In [70]:
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

马斯克的观点是AI将对人类文明产生深远的影响。他认为，随着人工智能技术的不断发展和应用，AI在许多领域中的应用将使人类社会发生翻天覆地的变化。
马斯克强调了AI在未来人类文明中可能产生的重大影响。首先，AI技术的发展可能会改变人类社会的生产方式和生活方式。例如，AI技术的发展可能会导致大量机器人的出现，从而极大地改变了人类社会的生产效率和生活方式。

其次，AI技术的发展可能会对人类社会的文化传统产生深远的影响。例如，AI技术的发展可能会导致大量计算机语言的出现，从而极大地改变了人类社会的文化传统。

再次，AI技术的发展可能会对人类社会的心理健康产生深远的影响。例如，AI技术的发展可能会导致大量人工智能模型的出现，从而极大地影响了人类社会的心理健康。

总之，马斯克的观点认为AI将在未来人类文明中产生深远的影响，包括改变人类社会的生产方式和生活方式、可能对人类社会的文化传统产生深远影响、可能对人类社会的心理健康产生深远影响等。因此，我们可以得出结论：随着AI技术的发展，AI在人类社会文化传统及心理健康方面将产生深远影响。


In [73]:
from langchain_core.runnables import RunnableParallel
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke(query)

{'context': [Document(page_content='我认为人工智能在未来人类的演进中将发挥重要作用，并对文明产生深远的影响。\n\n我们已经见证了数字计算能力的爆炸式增长。一个关键指标是数字计算机与生物大脑计算能力之比。这意味着人类与机器之间的算力比率不断增加，从而拉大了机器和生物之间的算力差距。\n\n随着时间的推移，人工智能在整体智能中所占比例将逐渐降低，相对于机器智能而言，这将是一个根本性的深刻变化。尽管现在很难完全理解其影响，但这可能是人类历史上最为深刻的时期之一。特斯拉的Optimus人型机器人目前仍处于早期开发阶段，但未来我们将会看到更多的机器人。\n\n因此，我们需要考虑另一个比例的问题，即机器人与人类的比例是多少？\n\n目前看来，这个比例将超过1:1，也就是说全球机器人的数量将超过人类的数量。机器人的计算能力要远远超过人类，这似乎是一个发展趋势。这将带来积极的影响，同时也会带来一些消极的影响。积极方面是我们将进入一个没有短缺的时代，几乎任何你想要的东西都能够立即获得。由于未来世界中将存在大量机器人，它们的生产效率将远远超过人类，这是一个非常深刻的变化。\n\n因此，我们需要小心确保最终结果对人类有益。', metadata={'source': "../example_data/Elon Musk's Speech at WAIC 2023.txt"}),
  Document(page_content='我认为人工智能在未来人类的演进中将发挥重要作用，并对文明产生深远的影响。\n\n我们已经见证了数字计算能力的爆炸式增长。一个关键指标是数字计算机与生物大脑计算能力之比。这意味着人类与机器之间的算力比率不断增加，从而拉大了机器和生物之间的算力差距。\n\n随着时间的推移，人工智能在整体智能中所占比例将逐渐降低，相对于机器智能而言，这将是一个根本性的深刻变化。尽管现在很难完全理解其影响，但这可能是人类历史上最为深刻的时期之一。特斯拉的Optimus人型机器人目前仍处于早期开发阶段，但未来我们将会看到更多的机器人。\n\n因此，我们需要考虑另一个比例的问题，即机器人与人类的比例是多少？\n\n目前看来，这个比例将超过1:1，也就是说全球机器人的数量将超过人类的数量。机器人的计算能力要远远超过人类，这似乎是一个发展趋势。这将带来积极的

In [75]:
for chunk in rag_chain_with_source.stream(query):
    print(chunk)

{'question': '马斯克认为AI将对人类文明产生什么影响'}
{'context': [Document(page_content='我认为人工智能在未来人类的演进中将发挥重要作用，并对文明产生深远的影响。\n\n我们已经见证了数字计算能力的爆炸式增长。一个关键指标是数字计算机与生物大脑计算能力之比。这意味着人类与机器之间的算力比率不断增加，从而拉大了机器和生物之间的算力差距。\n\n随着时间的推移，人工智能在整体智能中所占比例将逐渐降低，相对于机器智能而言，这将是一个根本性的深刻变化。尽管现在很难完全理解其影响，但这可能是人类历史上最为深刻的时期之一。特斯拉的Optimus人型机器人目前仍处于早期开发阶段，但未来我们将会看到更多的机器人。\n\n因此，我们需要考虑另一个比例的问题，即机器人与人类的比例是多少？\n\n目前看来，这个比例将超过1:1，也就是说全球机器人的数量将超过人类的数量。机器人的计算能力要远远超过人类，这似乎是一个发展趋势。这将带来积极的影响，同时也会带来一些消极的影响。积极方面是我们将进入一个没有短缺的时代，几乎任何你想要的东西都能够立即获得。由于未来世界中将存在大量机器人，它们的生产效率将远远超过人类，这是一个非常深刻的变化。\n\n因此，我们需要小心确保最终结果对人类有益。', metadata={'source': "../example_data/Elon Musk's Speech at WAIC 2023.txt"}), Document(page_content='我认为人工智能在未来人类的演进中将发挥重要作用，并对文明产生深远的影响。\n\n我们已经见证了数字计算能力的爆炸式增长。一个关键指标是数字计算机与生物大脑计算能力之比。这意味着人类与机器之间的算力比率不断增加，从而拉大了机器和生物之间的算力差距。\n\n随着时间的推移，人工智能在整体智能中所占比例将逐渐降低，相对于机器智能而言，这将是一个根本性的深刻变化。尽管现在很难完全理解其影响，但这可能是人类历史上最为深刻的时期之一。特斯拉的Optimus人型机器人目前仍处于早期开发阶段，但未来我们将会看到更多的机器人。\n\n因此，我们需要考虑另一个比例的问题，即机器人与人类的比例是多少？\n\n目前看来，这个比例将超过1:1，也就是说全球机器人的数量将超过人类的数量。

{'answer': '重要的'}
{'answer': '技术和'}
{'answer': '研发'}
{'answer': '挑战'}
{'answer': '和'}
{'answer': '机遇'}
{'answer': '。\n\n'}
{'answer': '最后'}
{'answer': '，'}
{'answer': '从'}
{'answer': '全'}
{'answer': '自动驾驶'}
{'answer': '汽车'}
{'answer': '的战略'}
{'answer': '规划'}
{'answer': '和发展'}
{'answer': '前景'}
{'answer': '的角度'}
{'answer': '来看'}
{'answer': '，'}
{'answer': '特斯拉'}
{'answer': '作为'}
{'answer': '人工智能'}
{'answer': '领域'}
{'answer': '的一个'}
{'answer': '重要'}
{'answer': '参与者'}
{'answer': '，在'}
{'answer': '全球'}
{'answer': '范围内'}
{'answer': '进行了'}
{'answer': '广泛的'}
{'answer': '测试'}
{'answer': '和'}
{'answer': '试'}
{'answer': '驾'}
{'answer': '，并'}
{'answer': '且'}
{'answer': '已经在'}
{'answer': '一些'}
{'answer': '特定'}
{'answer': '的城市'}
{'answer': '和地区'}
{'answer': '开始'}
{'answer': '进行'}
{'answer': '商业'}
{'answer': '运营'}
{'answer': '和'}
{'answer': '商业化'}
{'answer': '试点'}
{'answer': '，'}
{'answer': '但在'}
{'answer': '全'}
{'answer': '自动驾驶'}
{'answer': '汽车'}
{'answer': '的战略'}
{'answer': '规划'}
{'answer': '和发展'}
